In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE

In [122]:
df0=pd.read_csv("DDos_and_Normal_Traffic_Dataset.csv")
df0.shape

(53870, 28)

In [103]:
df=pd.read_csv("dataset.csv")

In [120]:
df.shape

(490567, 28)

In [89]:
print(df['label'].value_counts())

label
1    279155
0    211412
Name: count, dtype: int64


In [14]:
# Count missing value for each feature
print(df.isna().sum())

timestamp                   0
datapath_id                 0
flow_id                     0
ip_src                      0
tp_src                      0
ip_dst                      0
tp_dst                      0
ip_proto                    0
icmp_code                   0
icmp_type                   0
flow_duration_sec           0
flow_duration_nsec          0
idle_timeout                0
hard_timeout                0
flags                       0
packet_count                0
byte_count                  0
packet_count_per_second     0
packet_count_per_nsecond    0
byte_count_per_second       0
byte_count_per_nsecond      0
avg_packet_size             0
flow_duration_total         0
idle_mean                   0
idle_std                    0
idle_max                    0
idle_min                    0
label                       0
dtype: int64


In [5]:
df.head(5)

,timestamp,datapath_id,flow_id,ip_src,tp_src,ip_dst,tp_dst,ip_proto,icmp_code,icmp_type,...,packet_count_per_nsecond,byte_count_per_second,byte_count_per_nsecond,avg_packet_size,flow_duration_total,idle_mean,idle_std,idle_max,idle_min,label
0,1.719371e+09,1,192.168.0.15050192.168.1.4532686,192.168.0.1,5050,192.168.1.4,53268,6,-1,-1,...,8.863929e-05,2.635596e+06,5.850380e-03,66.002104,2.901,0.000000,0.0,0.000000,0.000000,0
1,1.719371e+09,1,192.168.1.453268192.168.0.150506,192.168.1.4,53268,192.168.0.1,5050,6,-1,-1,...,1.384392e-04,3.108656e+09,6.877558e+00,49679.278252,2.904,0.000000,0.0,0.000000,0.000000,0
2,1.719371e+09,2,192.168.0.15050192.168.1.4532686,192.168.0.1,5050,192.168.1.4,53268,6,-1,-1,...,9.024181e-05,2.635596e+06,5.956149e-03,66.002104,2.885,0.000815,0.0,0.000815,0.000815,0
3,1.719371e+09,2,192.168.1.453268192.168.0.150506,192.168.1.4,53268,192.168.0.1,5050,6,-1,-1,...,1.372259e-04,3.108656e+09,6.817228e+00,49678.882078,2.912,0.000815,0.0,0.000815,0.000815,0
4,1.719371e+09,2,192.168.1.40192.168.1.501,192.168.1.4,0,192.168.1.5,0,1,0,8,...,2.202643e-09,9.800000e+01,2.158590e-07,98.000000,2.908,0.000000,0.0,0.000000,0.000000,0


In [126]:
# Xóa các cột không cần thiết hoặc không phù hợp cho mô hình
cleaned_df = df.drop(['timestamp', 'datapath_id', 'flow_id', 'ip_src'], axis=1)
# # Replace dots in IP addresses to convert to numerical representation
cleaned_df['ip_src'] = cleaned_df['ip_src'].str.replace('.', '')
cleaned_df['ip_dst'] = cleaned_df['ip_dst'].str.replace('.', '')

In [127]:
# Shuffle dữ liệu
cleaned_df = cleaned_df.sample(frac=1, random_state=42).reset_index(drop=True)

# Stratified sampling để chia dữ liệu thành tập huấn luyện và kiểm tra
X = cleaned_df.drop('label', axis=1)
y = cleaned_df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Over-sampling using SMOTE
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Kiểm tra phân bố nhãn sau khi resampling
print(y_train_resampled.value_counts())


label
1    223324
0    223324
Name: count, dtype: int64


In [128]:
# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Standardized data
scaler = StandardScaler()
X_train_resampled = scaler.fit_transform(X_train_resampled)
X_test = scaler.transform(X_test)

In [129]:
from sklearn.neighbors import KNeighborsClassifier
model_dt = KNeighborsClassifier(n_neighbors=5, metric='minkowski', p=2, weights='distance')
model_dt.fit(X_train_resampled, y_train_resampled)
# Dự đoán trên tập kiểm tra
y_pred_dt = model_dt.predict(X_test)

# Đánh giá mô hình
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_dt))
print("Classification Report:\n", classification_report(y_test, y_pred_dt))
print("Accuracy Score:", accuracy_score(y_test, y_pred_dt))

Confusion Matrix:
 [[42186    97]
 [  127 55704]]
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     42283
           1       1.00      1.00      1.00     55831

    accuracy                           1.00     98114
   macro avg       1.00      1.00      1.00     98114
weighted avg       1.00      1.00      1.00     98114

Accuracy Score: 0.9977169415170108


In [130]:
from sklearn.model_selection import cross_val_score

cv_scores = cross_val_score(model_dt, X, y, cv=5)
print(f'Cross-validation scores: {cv_scores}')
print(f'Mean CV score: {cv_scores.mean()}')


Cross-validation scores: [0.99450639 0.99410889 0.99450633 0.99434326 0.99421076]
Mean CV score: 0.9943351266085345


In [132]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import learning_curve

train_sizes, train_scores, test_scores = learning_curve(model_dt, X, y, cv=5, n_jobs=-1, train_sizes=np.linspace(0.1, 1.0, 10))

# Tính toán giá trị trung bình và độ lệch chuẩn
train_scores_mean = train_scores.mean(axis=1)
train_scores_std = train_scores.std(axis=1)
test_scores_mean = test_scores.mean(axis=1)
test_scores_std = test_scores.std(axis=1)

plt.figure()
plt.title("Learning Curves (KNN)")
plt.xlabel("Training examples")
plt.ylabel("Score")
plt.grid()

plt.fill_between(train_sizes, train_scores_mean - train_scores_std, train_scores_mean + train_scores_std, alpha=0.1, color="r")
plt.fill_between(train_sizes, test_scores_mean - test_scores_std, test_scores_mean + test_scores_std, alpha=0.1, color="g")
plt.plot(train_sizes, train_scores_mean, 'o-', color="r", label="Training score")
plt.plot(train_sizes, test_scores_mean, 'o-', color="g", label="Cross-validation score")

plt.legend(loc="best")
plt.show()


KeyboardInterrupt: 

In [12]:
from sklearn.model_selection import GridSearchCV
# Khởi tạo mô hình KNN
knn = KNeighborsClassifier()

# Định nghĩa dải giá trị của các tham số để tìm kiếm
param_grid = {
    'n_neighbors': [3, 5, 7, 9],
    'weights': ['uniform', 'distance'],
    'metric': ['minkowski'],
    'p': [1, 2]
}

# Tạo đối tượng GridSearchCV
grid_search = GridSearchCV(estimator=knn, param_grid=param_grid, cv=5, n_jobs=-1, verbose=1)

# Huấn luyện mô hình với GridSearch
grid_search.fit(X_train_resampled, y_train_resampled)

# In ra các tham số tốt nhất và điểm số hiệu suất
print("Best parameters found: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

Fitting 5 folds for each of 16 candidates, totalling 80 fits
Best parameters found:  {'metric': 'minkowski', 'n_neighbors': 3, 'p': 2, 'weights': 'distance'}
Best score:  0.9473567703494105
